In [ ]:
from huawei import router1, router2, HuaweiRouter
from pathlib import Path

log_dir = Path("log")
config_router1 = Path("router1.txt")
config_router2 = Path("router2.txt")

In [ ]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.sysname("Router1")
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.sysname("Router2")

In [ ]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.configure(config_router1)
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.configure(config_router2)

In [ ]:
ospf_commands = [
    "display ospf lsdb router",
    "display ospf lsdb network",
    "display ospf lsdb",
    "display ospf peer"
]
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands(ospf_commands)
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands(ospf_commands)

In [ ]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands("display interface brief")
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands("display interface brief")